In [0]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [0]:
from __future__ import print_function
import keras

from keras.layers import Conv2D, MaxPool2D, Input, ZeroPadding2D
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import print_summary, to_categorical
from keras import backend as K
import sys
import os
import numpy as np
import pickle
import h5py

Using TensorFlow backend.


###Load Dataset

In [0]:
num_classes = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar100.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### Define the Model

In [0]:
# define cnn model
 model = Sequential()

 model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
 model.add(Activation('elu'))
 model.add(Conv2D(128, (3, 3)))
 model.add(Activation('elu'))
 model.add(MaxPooling2D(pool_size=(2, 2)))
 #model.add(Dropout(0.25))

 model.add(Conv2D(256, (3, 3), padding='same'))
 model.add(Activation('elu'))
 model.add(Conv2D(256, (3, 3)))
 model.add(Activation('elu'))
 model.add(MaxPooling2D(pool_size=(2, 2)))
 model.add(Dropout(0.25))

 model.add(Conv2D(512, (3, 3), padding='same'))
 model.add(Activation('elu'))
 model.add(Conv2D(512, (3, 3)))
 model.add(Activation('elu'))
 model.add(MaxPooling2D(pool_size=(2, 2)))
 model.add(Dropout(0.25))

 model.add(Flatten())
 model.add(Dense(1024))
 model.add(Activation('elu'))
 model.add(Dropout(0.5))
 model.add(Dense(num_classes))
 model.add(Activation('softmax'))

 # initiate RMSprop optimizer
 opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

 # Let's train the model using RMSprop
 model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy']) 






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




### Data Augmentation running for 100 epochs

In [0]:
epochs = 100
data_augmentation = True
num_predictions = 20
batch_size = 64
validations = []

for i in range(epochs):
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=1)#,
                            #validation_data=(x_test, y_test))
history = validations.append(model.evaluate_generator(datagen.flow(x_test, y_test,
                                          batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size))

pickle.dump(validations, open("loss_validation.p",'wb'))

Using real-time data augmentation.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1





781/781 [==============================] - 35s 45ms/step - loss: 3.9850 - acc: 0.0964
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 39ms/step - loss: 3.4577 - acc: 0.1844
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 38ms/step - loss: 3.1690 - acc: 0.2384
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 39ms/step - loss: 2.9845 - acc: 0.2747
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 39ms/step - loss: 2.8478 - acc: 0.2996
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 39ms/step - loss: 2.7477 - acc: 0.3213
Using real-time data augmentation.
Epoch 1/1
781/781 [==============================] - 30s 39ms/step - loss: 2.

### Loading label names to use in prediction results

In [0]:
label_list_path = 'datasets/cifar-100-python/meta'

keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

### Evaluating and Predicting the Final Model

In [0]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['fine_label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['fine_label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.65
Actual Label = mountain vs. Predicted Label = can
Actual Label = forest vs. Predicted Label = skunk
Actual Label = seal vs. Predicted Label = wolf
Actual Label = mushroom vs. Predicted Label = otter
Actual Label = sea vs. Predicted Label = aquarium_fish
Actual Label = tulip vs. Predicted Label = cattle
Actual Label = camel vs. Predicted Label = castle
Actual Label = butterfly vs. Predicted Label = wolf
Actual Label = cloud vs. Predicted Label = lamp
Actual Label = apple vs. Predicted Label = lawn_mower
Actual Label = sea vs. Predicted Label = butterfly
Actual Label = skunk vs. Predicted Label = dolphin
Actual Label = streetcar vs. Predicted Label = crab
Actual Label = rocket vs. Predicted Label = cup
Actual Label = lamp vs. Predicted Label = orchid
Actual Label = lion vs. Predicted Label = crocodile
Actual Label = tulip vs. Predicted Label = bridge
Actual Label = wolf vs. Predicted Label = lizard
Actual Label = rose vs. Predicted Label = worm
Actual Label = orange